<a href="https://colab.research.google.com/github/vilsonrodrigues/TensorRT-Python-Plugin-Triton-Kernels/blob/main/tensorrt_python_plugin_triton_kernel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://towardsdatascience.com/writing-custom-cuda-kernels-with-triton-abf6b6ad1168
# https://ai.lefebvre-sarrut.eu/2023/07/20/deep-dive-into-kernel-fusion-accelerating-inference-in-llama-v2/#openai-triton-rewriting

In [2]:
!nvidia-smi

Mon Jul 22 21:42:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   48C    P8              13W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
!pip install -U torch tensorrt cuda-python

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 55.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cac

In [3]:
!pip install onnx onnx_graphsurgeon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 9.1 MB/s eta 0:00:00


In [4]:
import ctypes
from typing import List
import numpy as np
import tensorrt as trt
import torch
from cuda import cudart
import triton
import triton.language as tl

@triton.jit
def add_kernel(x_ptr,  # *Pointer* to first input vector.
               y_ptr,  # *Pointer* to second input vector.
               output_ptr,  # *Pointer* to output vector.
               n_elements,  # Size of the vector.
               BLOCK_SIZE: tl.constexpr,  # Number of elements each program should process.
               # NOTE: `constexpr` so it can be used as a shape value.
               ):
    # There are multiple 'programs' processing different data. We identify which program
    # we are here:
    pid = tl.program_id(0)  # tl.program_id(0) -> blockIdx.x
    # This program will process inputs that are offset from the initial data.
    # For instance, if you had a vector of length 256 and block_size of 64, the programs
    # would each access the elements [0:64, 64:128, 128:192, 192:256].
    # Note that offsets is a list of pointers:
    block_start = pid * BLOCK_SIZE
    offsets = block_start + tl.arange(0, BLOCK_SIZE)
    # Create a mask to guard memory operations against out-of-bounds accesses.
    mask = offsets < n_elements
    # Load x and y from DRAM, masking out any extra elements in case the input is not a
    # multiple of the block size.
    x = tl.load(x_ptr + offsets, mask=mask)
    y = tl.load(y_ptr + offsets, mask=mask)
    output = x + y
    # Write x + y back to DRAM.
    tl.store(output_ptr + offsets, output, mask=mask)

# Let's also declare a helper function to (1) allocate the `c` tensor
# and (2) enqueue the above kernel with appropriate grid/block sizes:
def add_layer(a: torch.Tensor, b: torch.Tensor):
    # We need to preallocate the output.
    output = torch.empty_like(a)
    assert a.is_cuda and b.is_cuda and output.is_cuda
    n_elements = output.numel()
    # The SPMD launch grid denotes the number of kernel instances that run in parallel.
    # It is analogous to CUDA launch grids. It can be either Tuple[int], or Callable(metaparameters) -> Tuple[int].
    # In this case, we use a 1D grid where the size is the number of blocks:
    grid = lambda meta: (triton.cdiv(n_elements, meta['BLOCK_SIZE']), )
    # NOTE:
    #  - Each torch.tensor object is implicitly converted into a pointer to its first element.
    #  - `triton.jit`'ed functions can be indexed with a launch grid to obtain a callable GPU kernel.
    #  - Don't forget to pass meta-parameters as keywords arguments.
    add_kernel[grid](a, b, output, n_elements, BLOCK_SIZE=256)
    # We return a handle to z but, since `torch.cuda.synchronize()` hasn't been called, the kernel is still
    # running asynchronously at this point.
    return output

class AddPlugin(trt.IPluginV3, trt.IPluginV3OneCore, trt.IPluginV3OneBuild, trt.IPluginV3OneRuntime):

    def __init__(self, field_collections=None):
        trt.IPluginV3.__init__(self)
        trt.IPluginV3OneCore.__init__(self)
        trt.IPluginV3OneBuild.__init__(self)
        trt.IPluginV3OneRuntime.__init__(self)
        self.plugin_name = "AddPlugin"  # necessary as function `getPluginType` in C++
        self.plugin_version = "1"  # necessary as function `getPluginVersion` in C++
        self.num_outputs = 1  # necessary as function `getNbOutputs` in C++
        self.plugin_namespace = ""  # necessary as function `setPluginNamespace`/ `getPluginNamespace` in C++
        self.device = 0  # default device is cuda:0, can be get by `cuda.cuDeviceGet(0)`
        return

    def get_capability_interface(self, plugin_capability_type: trt.PluginCapabilityType) -> trt.IPluginCapability:
        return self

    def clone(self) -> trt.IPluginV3:
        cloned_plugin = AddPlugin()
        cloned_plugin.__dict__.update(self.__dict__)
        return cloned_plugin

    def configure_plugin(self, dptd_in: List[trt.DynamicPluginTensorDesc], dptd_out: List[trt.DynamicPluginTensorDesc]) -> None:
        """ Configure the plugin based-on inputs and outputs

        No required to this plugin
        Example: https://github.com/NVIDIA/TensorRT/blob/release/10.2/samples/python/python_plugin/circ_pad_plugin_triton.py#L125
        """
        return

    def get_output_data_types(self, input_types: List[trt.DataType]) -> List[trt.DataType]:
        """ Describe the output data type based-on inputs

        In this plugin the output data type is the same than input because
        add op no change the data type
        """
        return [input_types[0]]

    def get_output_shapes(self, inputs: List[trt.DimsExprs], shape_inputs: List[trt.DimsExprs], expr_builder: trt.IExprBuilder) -> List[trt.DimsExprs]:
        """ Describe the output shape based-on inputs

        In this plugin the output is the same than input because
        add op no change the shape
        """
        output_dims = trt.DimsExprs(inputs[0])
        return [output_dims]

    def supports_format_combination(self, pos: int, in_out: List[trt.DynamicPluginTensorDesc], num_inputs: int) -> bool:
        """ Check if the data format is supported by plugin

        Args:
            pos: tensor position (in/out)
            in_out: dynamic tensor description
            num_inputs: number of inputs

        Returns:
            True if supported, False if not
        """
        assert num_inputs > 1
        assert pos < len(in_out)

        res = False
        desc = in_out[pos].desc
        # check inputs
        if pos in [0, 1]:
            # inputs should be float16 or float32
            res = (desc.type == trt.float32 or desc.type == trt.float16) and desc.format == trt.TensorFormat.LINEAR
        # check output
        elif pos == 2:
            # output should have the same type as the input
            res = desc.type == in_out[0].desc.type and desc.format == trt.TensorFormat.LINEAR
        if False:  # print information about the input / output
            info = f"    {pos=}:"
            info += f"{[str(in_out[i].type)[9:] for i in range(len(in_out))]},"
            info += f"{[str(in_out[i].format)[13:] for i in range(len(in_out))]}"
            info += f"->{res=}"
            print(info)
        return res

    def get_workspace_size(self, dptd_in: List[trt.DynamicPluginTensorDesc], dptd_out: List[trt.DynamicPluginTensorDesc]) -> int:
        """ Return necessary workspace to this plugin """
        return 0

    def get_valid_tactics(self) -> List[int]:
        """ Return a list from valid tatics """
        return [1]

    def set_tactic(self: trt.IPluginV3, tactic: int) -> None:
        """ Define a tatic to be used to this plugin """
        return None

    def on_shape_change(self, ptd_in: List[trt.PluginTensorDesc], ptd_out: List[trt.PluginTensorDesc]) -> ModuleNotFoundError:
        """ Method called when there is a chance in tensors """
        return None

    def attach_to_context(self, resource_context: trt.IPluginResourceContext) -> trt.IPluginV3:
        """ Attach this plugin in a context """
        return self.clone()

    def get_fields_to_serialize(self) -> trt.PluginFieldCollection:
        """ Returns the fields to serialize

        To this plugin no field is implemented
        """
        field_collection = trt.PluginFieldCollection([])
        return field_collection

    def enqueue(self, input_desc: List[trt.PluginTensorDesc], output_desc: List[trt.PluginTensorDesc], inputs: List[int], outputs: List[int], workspace: int, stream: int) -> None:
        """ Execute kernel on GPU

        Implement C = A + B using OpenAI Triton

        Args:
            input_desc: input tensor description list
            output_desc: output tensor description list
            inputs: pointer list to inputs
            outputs: pointer list to outputs
            workspace
            stream
        """
        A_DATA_TYPE = trt.nptype(input_desc[0].type)
        B_DATA_TYPE = trt.nptype(input_desc[1].type)
        assert A_DATA_TYPE == B_DATA_TYPE

        A_SHAPE = tuple(input_desc[0].dims)
        B_SHAPE = tuple(input_desc[1].dims)
        assert A_SHAPE == B_SHAPE

        A_N_ELEMENT = np.prod(A_SHAPE)
        B_N_ELEMENT = np.prod(B_SHAPE)

        # pointer -> numpy.ndarray -> torch.Tensor
        BUFFER_SIZE = A_N_ELEMENT * np.dtype(A_DATA_TYPE).itemsize
        cpp_data_type = ctypes.c_int16 if A_DATA_TYPE == np.float16 else ctypes.c_float

        A_POINTER = ctypes.cast(inputs[0], ctypes.POINTER(cpp_data_type * A_N_ELEMENT))[0]
        A_NP_ARRAY = np.ndarray(A_SHAPE, dtype=A_DATA_TYPE, buffer=A_POINTER)
        A_TENSOR = torch.as_tensor(A_NP_ARRAY, device="cuda")

        B_POINTER = ctypes.cast(inputs[1], ctypes.POINTER(cpp_data_type * B_N_ELEMENT))[0]
        B_NP_ARRAY = np.ndarray(B_SHAPE, dtype=B_DATA_TYPE, buffer=B_POINTER)
        B_TENSOR = torch.as_tensor(B_NP_ARRAY, device="cuda")

        # C = A + B with triton add layer
        C_TENSOR = add_layer(A_TENSOR, B_TENSOR)

        # copy add output to buffer
        kind = cudart.cudaMemcpyKind.cudaMemcpyDeviceToDevice
        cudart.cudaMemcpyAsync(outputs[0], C_TENSOR.data_ptr(), BUFFER_SIZE, kind, stream)
        return

class AddPluginCreator(trt.IPluginCreatorV3One):

    def __init__(self):
        trt.IPluginCreatorV3One.__init__(self)
        self.name = "AddPlugin"  # necessary as function `getPluginName` in C++
        self.plugin_version = "1"  # necessary as function `getPluginVersion` in C++
        self.plugin_namespace = ""  # necessary as function `setPluginNamespace`/ `getPluginNamespace` in C++
        self.field_names = trt.PluginFieldCollection([])
        return

    def create_plugin(self, name: str, field_collection: trt.PluginFieldCollection, phase: trt.TensorRTPhase):
        return AddPlugin()

def datatype_np_to_torch(datatype_np):
    if datatype_np == np.float32:
        return torch.float32
    if datatype_np == np.float16:
        return torch.float16
    if datatype_np == np.int8:
        return torch.int8
    if datatype_np == np.int32:
        return torch.int32
    if datatype_np == bool:
        return torch.bool
    if datatype_np == np.uint8:
        return torch.uint8
    if datatype_np == np.int64:
        return torch.int64
    return None

if __name__ == "__main__":
    from collections import OrderedDict
    import onnx
    import onnx_graphsurgeon as gs
    import numpy as np

    onnx_path = "add_model.onnx"

    # create input variables with batch fixed = 1
    input_A = gs.Variable(name="A", dtype=np.float32, shape=(1, "N"))
    input_B = gs.Variable(name="B", dtype=np.float32, shape=(1, "N"))

    # create output variable
    output = gs.Variable(name="C", dtype=np.float32)

    # create node with custom plugin op
    add_plugin_node = gs.Node(
        name="AddPlugin",
        op="AddPlugin",
        inputs=[input_A, input_B],
        outputs=[output],
        #attrs={"pads": pads},
    )

    graph = gs.Graph(nodes=[add_plugin_node], inputs=[input_A, input_B], outputs=[output], opset=17)
    onnx.save(gs.export_onnx(graph), onnx_path)

    # build engine
    TRT_LOGGER = trt.Logger(trt.Logger.VERBOSE)

    # register plugin creator
    plg_registry = trt.get_plugin_registry()
    my_plugin_creator = AddPluginCreator()
    plg_registry.register_creator(my_plugin_creator, "")

    builder = trt.Builder(TRT_LOGGER)
    config = builder.create_builder_config()
    # Set cache
    cache = config.create_timing_cache(b"")
    config.set_timing_cache(cache, ignore_mismatch=False)
    flag = 1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)
    network = builder.create_network(flag)
    parser = trt.OnnxParser(network, TRT_LOGGER)
    with open(onnx_path, "rb") as f:
        if not parser.parse(f.read()):
            print(f"ERROR: Failed to parse the ONNX file {onnx_path}")
            for error in range(parser.num_errors):
                print(parser.get_error(error))
    inputs = [network.get_input(i) for i in range(network.num_inputs)]
    outputs = [network.get_output(i) for i in range(network.num_outputs)]

    # set opt profile
    profile = builder.create_optimization_profile()
    # define range
    min_shape = [1] + [1]
    opt_shape = [1] + [1000]
    max_shape = [1] + [10000]
    for input in inputs:
        profile.set_shape(input.name, min_shape, opt_shape, max_shape)
    config.add_optimization_profile(profile)

    engine_bytes = builder.build_serialized_network(network, config)
    engine_path = "add_model.engine"
    with open(engine_path, "wb") as f:
        f.write(engine_bytes)

    runtime = trt.Runtime(TRT_LOGGER)
    with open(engine_path, "rb") as plan:
        engine = runtime.deserialize_cuda_engine(plan.read())

    # define inputs
    A = np.ones((1, 3))
    B = np.ones((1, 3))

    # create stream
    err, stream = cudart.cudaStreamCreate()

    # create Execution Context from the engine (analogy to a GPU context, or a CPU process)
    context = engine.create_execution_context()

    # get i/o tensors names
    tensor_name_list = [engine.get_tensor_name(i) for i in range(engine.num_io_tensors)]

    # set runtime size of input tensor if using Dynamic-Shape mode
    context.set_input_shape(tensor_name_list[0], A.shape)
    context.set_input_shape(tensor_name_list[1], B.shape)

    # Print information of input / output tensors
    for name in tensor_name_list:
        mode = engine.get_tensor_mode(name)
        data_type = engine.get_tensor_dtype(name)
        buildtime_shape = engine.get_tensor_shape(name)
        runtime_shape = context.get_tensor_shape(name)
        print(f"{'Input ' if mode == trt.TensorIOMode.INPUT else 'Output'}->{data_type}, {buildtime_shape}, {runtime_shape}, {name}")

    device = "cuda:0"
    # prepare the memory buffer on host and device
    buffer = OrderedDict()
    for name in tensor_name_list:
        data_type = engine.get_tensor_dtype(name)
        runtime_shape = context.get_tensor_shape(name)
        buffer[name] = torch.empty(tuple(runtime_shape), dtype=datatype_np_to_torch(data_type), device=device)

    # model input
    data_A = torch.ones((1,3), dtype=torch.float32)
    data_B = torch.ones((1,3), dtype=torch.float32)

    # set runtime data, MUST use contiguous, it is a SERIOUS lesson
    buffer[tensor_name_list[0]] = data_A.contiguous().to(device)
    buffer[tensor_name_list[1]] = data_B.contiguous().to(device)

    for name in tensor_name_list:
        # bind address of buffer to context
        context.set_tensor_address(name, buffer[name].data_ptr())

    # do inference computation
    context.execute_async_v3(stream)

    # synchronize to get outputs
    cudart.cudaStreamSynchronize(stream)
    for name in tensor_name_list:
        print(name)
        print(buffer[name])

Input ->DataType.FLOAT, (1, -1), (1, 3), A
Input ->DataType.FLOAT, (1, -1), (1, 3), B
Output->DataType.FLOAT, (1, -1), (1, 3), C
A
tensor([[1., 1., 1.]], device='cuda:0')
B
tensor([[1., 1., 1.]], device='cuda:0')
C
tensor([[2., 2., 2.]], device='cuda:0')


In [5]:
!pip freeze

absl-py==1.4.0
accelerate==0.32.1
aiohttp==3.9.5
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.15.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.11.1
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
branca==0.7.2
build==1.2.1
CacheControl==0.14.0
cachetools==5.4.0
catalogue==2.0.10
certifi==2024.7.4
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.86
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.18.1
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.4
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.5
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.1
cryptography==42.0.8
cuda-python==12.5.0
cudf-cu12 @ h